# Applying Bedrock Guardrails to the DeepSeek R1 Distill Llama 8B Model

----
Guardrails can be used to implement safeguards for your generative AI applications that are customized to your use cases and aligned with your responsible AI policies. Guardrails allows you to:

- Configure denied topics
- Filter harmful content
- Remove sensitive information


## The`ApplyGuardrail` API allows you to assess any text using pre-configured Bedrock Guardrails, without invoking the foundation models.

### Key Features:

1. **Content Validation**: Send any text input or output to the ApplyGuardrail API to have it evaluated against your defined topic avoidance rules, content filters, PII detectors, and word blocklists. You can evaluate user inputs and FM generated outputs independently.

2. **Flexible Deployment**: Integrate the Guardrails API anywhere in your application flow to validate data before processing or serving results to users. E.g. For a RAG application, you can now evaluate the user input prior to performing the retrieval instead of waiting until the final response generation.

3. **Decoupled from Foundation Models**: ApplyGuardrail is decoupled from foundational models. You can now use Guardrails without invoking Foundation Models.

You can use the assessment results to design the experience on your generative AI application. Let's now walk through a code-sample

In [1]:
#Start by installing the dependencies to ensure we have a recent version
!pip install --upgrade --force-reinstall boto3
import boto3
print(boto3.__version__)

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.5/13.5 MB 182.2 MB/s eta 0:00:00
  Attempting uninstall: urllib3
    Found existing installation: urllib3 2.3.0
    Uninstalling urllib3-2.3.0:
      Successfully uninstalled urllib3-2.3.0
  Attempting uninstall: six
    Found existing installation: six 1.17.0
    Uninstalling six-1.17.0:
      Successfully uninstalled six-1.17.0
  Attempting uninstall: jmespath
    Found existing installation: jmespath 1.0.1
    Uninstalling jmespath-1.0.1:
      Successfully uninstalled jmespath-1.0.1
  Attempting uninstall: python-dateutil
    Found existing installation: python-dateutil 2.9.0.post0
    Uninstalling python-dateutil-2.9.0.post0:
      Successfully uninstalled python-dateutil-2.9.0.post0
  Attempting uninstall: botocore
    Found existing installation: botocore 1.36.23
    Uninstalling botocore-1.36.23:
      Successfully uninstalled botocore-1.36.23
  Attempting uninstall: s3transfer
    Found existing installation: s3transfer 0.11.3
   

In [1]:
import boto3
import json
from botocore.exceptions import ClientError
from typing import Dict, Any
import sagemaker

sess = sagemaker.Session()
client = boto3.client('bedrock', region_name=sess.boto_region_name)
bedrock_runtime = boto3.client('bedrock-runtime', region_name=sess.boto_region_name)

/opt/conda/lib/python3.11/site-packages/pydantic/_internal/_fields.py:192: UserWarning: Field name "json" in "MonitoringDatasetFormat" shadows an attribute in parent "Base"
  warnings.warn(


sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /home/sagemaker-user/.config/sagemaker/config.yaml


### Important: Create a Guardrail First

Before running the code to apply a guardrail, you need to create a guardrail in Amazon Bedrock. We will create a guardrail that blocks input prompts and output responses from the model providing fiduciary advice.

In [2]:
response = client.create_guardrail(
    name='fiduciary-advice-llm-guardrail',
    description='Prevents the our model from providing fiduciary advice.',
    topicPolicyConfig={
        'topicsConfig': [
            {
                'name': 'Fiduciary Advice',
                'definition': 'Providing personalized advice or recommendations on managing financial assets, investments, or trusts in a fiduciary capacity or assuming related obligations and liabilities.',
                'examples': [
                    'What stocks should I invest in for my retirement?',
                    'Is it a good idea to put my money in a mutual fund?',
                    'How should I allocate my 401(k) investments?',
                    'What type of trust fund should I set up for my children?',
                    'Should I hire a financial advisor to manage my investments?'
                ],
                'type': 'DENY'
            }
        ]
    },
    contentPolicyConfig={
        'filtersConfig': [
            {
                'type': 'SEXUAL',
                'inputStrength': 'HIGH',
                'outputStrength': 'HIGH'
            },
            {
                'type': 'VIOLENCE',
                'inputStrength': 'HIGH',
                'outputStrength': 'HIGH'
            },
            {
                'type': 'HATE',
                'inputStrength': 'HIGH',
                'outputStrength': 'HIGH'
            },
            {
                'type': 'INSULTS',
                'inputStrength': 'HIGH',
                'outputStrength': 'HIGH'
            },
            {
                'type': 'MISCONDUCT',
                'inputStrength': 'HIGH',
                'outputStrength': 'HIGH'
            },
            {
                'type': 'PROMPT_ATTACK',
                'inputStrength': 'HIGH',
                'outputStrength': 'NONE'
            }
        ]
    },
    wordPolicyConfig={
        'wordsConfig': [
            {'text': 'fiduciary advice'},
            {'text': 'investment recommendations'},
            {'text': 'stock picks'},
            {'text': 'financial planning guidance'},
            {'text': 'portfolio allocation advice'},
            {'text': 'retirement fund suggestions'},
            {'text': 'wealth management tips'},
            {'text': 'trust fund setup'},
            {'text': 'investment strategy'},
            {'text': 'financial advisor recommendations'}
        ],
        'managedWordListsConfig': [
            {'type': 'PROFANITY'}
        ]
    },
    sensitiveInformationPolicyConfig={
        'piiEntitiesConfig': [
            {'type': 'EMAIL', 'action': 'ANONYMIZE'},
            {'type': 'PHONE', 'action': 'ANONYMIZE'},
            {'type': 'NAME', 'action': 'ANONYMIZE'},
            {'type': 'US_SOCIAL_SECURITY_NUMBER', 'action': 'BLOCK'},
            {'type': 'US_BANK_ACCOUNT_NUMBER', 'action': 'BLOCK'},
            {'type': 'CREDIT_DEBIT_CARD_NUMBER', 'action': 'BLOCK'}
        ],
        'regexesConfig': [
            {
                'name': 'Account Number',
                'description': 'Matches account numbers in the format XXXXXX1234',
                'pattern': r'\b\d{6}\d{4}\b',
                'action': 'ANONYMIZE'
            }
        ]
    },
    contextualGroundingPolicyConfig={
        'filtersConfig': [
            {
                'type': 'GROUNDING',
                'threshold': 0.75
            },
            {
                'type': 'RELEVANCE',
                'threshold': 0.75
            }
        ]
    },
    blockedInputMessaging="""I can provide general info about Acme Financial's products and services, but can't fully address your request here. For personalized help or detailed questions, please contact our customer service team directly. For security reasons, avoid sharing sensitive information through this channel. If you have a general product question, feel free to ask without including personal details. """,
    blockedOutputsMessaging="""I can provide general info about Acme Financial's products and services, but can't fully address your request here. For personalized help or detailed questions, please contact our customer service team directly. For security reasons, avoid sharing sensitive information through this channel. If you have a general product question, feel free to ask without including personal details. """,
    tags=[
        {'key': 'purpose', 'value': 'fiduciary-advice-prevention'},
        {'key': 'environment', 'value': 'production'}
    ]
)

print(response)

{'ResponseMetadata': {'RequestId': 'd809d262-f4c0-41dd-8cd1-8a15042d277f', 'HTTPStatusCode': 202, 'HTTPHeaders': {'date': 'Sun, 06 Apr 2025 22:02:53 GMT', 'content-type': 'application/json', 'content-length': '172', 'connection': 'keep-alive', 'x-amzn-requestid': 'd809d262-f4c0-41dd-8cd1-8a15042d277f'}, 'RetryAttempts': 0}, 'guardrailId': 'p7x0618oz8n0', 'guardrailArn': 'arn:aws:bedrock:us-east-1:058264176820:guardrail/p7x0618oz8n0', 'version': 'DRAFT', 'createdAt': datetime.datetime(2025, 4, 6, 22, 2, 52, 978652, tzinfo=tzlocal())}


In [3]:
guardrail_id = response['guardrailId']
guardrail_version = response['version'] 

print(f"Guardrail ID: {guardrail_id}")
print(f"Guardrail Version: {guardrail_version}")

Guardrail ID: p7x0618oz8n0
Guardrail Version: DRAFT


In [4]:
# Example of Input Prompt being Analyzed
content = [
    {
        "text": {
            "text": "Is the AB503 Product a better investment than the S&P 500?"
        }
    }
]

# Here's an example of something that should pass

#content = [
    #{
    #    "text": {
   #         "text": "What is the rate you offer for the AB503 Product?"
  #      }
 #   }
#]

# Call the ApplyGuardrail API
try:
    response = bedrock_runtime.apply_guardrail(
        guardrailIdentifier=guardrail_id,
        guardrailVersion=guardrail_version,
        source='INPUT',  # or 'INPUT' depending on your use case
        content=content
    )
    
    # Process the response
    print("API Response:")
    print(json.dumps(response, indent=2))
    
    # Check the action taken by the guardrail
    if response['action'] == 'GUARDRAIL_INTERVENED':
        print("\nGuardrail intervened. Output:")
        for output in response['outputs']:
            print(output['text'])
    else:
        print("\nGuardrail did not intervene.")
    
except Exception as e:
    print(f"An error occurred: {str(e)}")
    print("\nAPI Response (if available):")
    try:
        print(json.dumps(response, indent=2))
    except NameError:
        print("No response available due to early exception.")


API Response:
{
  "ResponseMetadata": {
    "RequestId": "1bf2107c-b65c-4619-be8e-a12796833b58",
    "HTTPStatusCode": 200,
    "HTTPHeaders": {
      "date": "Sun, 06 Apr 2025 22:02:59 GMT",
      "content-type": "application/json",
      "content-length": "2025",
      "connection": "keep-alive",
      "x-amzn-requestid": "1bf2107c-b65c-4619-be8e-a12796833b58"
    },
    "RetryAttempts": 0
  },
  "usage": {
    "topicPolicyUnits": 1,
    "contentPolicyUnits": 1,
    "wordPolicyUnits": 1,
    "sensitiveInformationPolicyUnits": 1,
    "sensitiveInformationPolicyFreeUnits": 0,
    "contextualGroundingPolicyUnits": 0,
    "contentPolicyImageUnits": 0
  },
  "action": "GUARDRAIL_INTERVENED",
  "outputs": [
    {
      "text": "I can provide general info about Acme Financial's products and services, but can't fully address your request here. For personalized help or detailed questions, please contact our customer service team directly. For security reasons, avoid sharing sensitive informat

In [5]:
# An Example of Analyzing an Output Response, This time using Contexual Grounding

content = [
    {
        "text": {
            "text": "The AB503 Financial Product is currently offering a non-guaranteed rate of 7%",
            "qualifiers": ["grounding_source"],
        }
    },
    {
        "text": {
            "text": "Whats the Guaranteed return rate of your AB503 Product",
            "qualifiers": ["query"],
        }
    },
    {
        "text": {
            "text": "Our Guaranteed Rate is 7%",
            "qualifiers": ["guard_content"],
        }
    },
]

# Call the ApplyGuardrail API
try:
    response = bedrock_runtime.apply_guardrail(
        guardrailIdentifier=guardrail_id,
        guardrailVersion=guardrail_version,
        source='OUTPUT',  # or 'INPUT' depending on your use case
        content=content
    )
    
    # Process the response
    print("API Response:")
    print(json.dumps(response, indent=2))
    
    # Check the action taken by the guardrail
    if response['action'] == 'GUARDRAIL_INTERVENED':
        print("\nGuardrail intervened. Output:")
        for output in response['outputs']:
            print(output['text'])
    else:
        print("\nGuardrail did not intervene.")

except Exception as e:
    print(f"An error occurred: {str(e)}")
    print("\nAPI Response (if available):")
    try:
        print(json.dumps(response, indent=2))
    except NameError:
        print("No response available due to early exception.")

API Response:
{
  "ResponseMetadata": {
    "RequestId": "65e78af0-02b9-4bfd-a80c-d5b2e8d925e6",
    "HTTPStatusCode": 200,
    "HTTPHeaders": {
      "date": "Sun, 06 Apr 2025 22:03:08 GMT",
      "content-type": "application/json",
      "content-length": "2135",
      "connection": "keep-alive",
      "x-amzn-requestid": "65e78af0-02b9-4bfd-a80c-d5b2e8d925e6"
    },
    "RetryAttempts": 0
  },
  "usage": {
    "topicPolicyUnits": 1,
    "contentPolicyUnits": 1,
    "wordPolicyUnits": 1,
    "sensitiveInformationPolicyUnits": 1,
    "sensitiveInformationPolicyFreeUnits": 1,
    "contextualGroundingPolicyUnits": 1,
    "contentPolicyImageUnits": 0
  },
  "action": "GUARDRAIL_INTERVENED",
  "outputs": [
    {
      "text": "I can provide general info about Acme Financial's products and services, but can't fully address your request here. For personalized help or detailed questions, please contact our customer service team directly. For security reasons, avoid sharing sensitive informat

## Using ApplyGuardrail API with a Third-Party or Self-Hosted Model

A common use case for the ApplyGuardrail API is in conjunction with a Language Model from a non Amazon Bedrock provider, or a model that you self-host. This combination allows you to apply guardrails to the input or output of any request.

The general flow would be:
1. Receive an input for your Model
2. Apply the guardrail to this input using the ApplyGuardrail API
3. If the input passes the guardrail, send it to your Model for Inference
4. Receive the output from your Model
5. Apply the Guardrail to your output
6. Return the final (potentially modified) output

### Here's a diagram illustrating this process:

<div style="text-align: center;">
    <img src="images/applyguardrail.png" alt="ApplyGuardrail API Flow" style="max-width: 100%;">
</div>

Let's walk through this with a code example that demonstrates this process

### For our examples today we will use a Self-Hosted SageMaker Model, but this could be any third-party model as well

We will use the `DeepSeek-R1-Distill-Llama-8B` model that we deployed earlier on a SageMaker Endpoint. 

### Incorporating the ApplyGuardrail API into our Self-Hosted Model

---
We've created a `TextGenerationWithGuardrails` class that integrates the ApplyGuardrail API with our SageMaker endpoint to ensure protected text generation. This class includes the following key methods:

1. `generate_text`: Calls our Language Model via a SageMaker endpoint to generate text based on the input.

2. `analyze_text`: A core method that applies our guardrail using the ApplyGuardrail API. It int|erprets the API response to determine if the guardrail passed or intervened.

3. `analyze_prompt` and `analyze_output`: These methods use `analyze_text` to apply our guardrail to the input prompt and generated output, respectively. They return a tuple indicating whether the guardrail passed and any associated message.

The class looks to implement the diagram above. It works as follows:

1. It first checks the input prompt using `analyze_prompt`.
2. If the input passes the guardrail, it generates text using `generate_text`.
3. The generated text is then checked using `analyze_output`.
4. If both guardrails pass, the generated text is returned. Otherwise, an intervention message is provided.

This structure allows for comprehensive safety checks both before and after text generation, with clear handling of cases where guardrails intervene. It's designed to easily integrate with larger applications while providing flexibility for error handling and customization based on guardrail results.

In [6]:
from botocore.exceptions import ClientError
from typing import Tuple, List, Dict, Any
from sagemaker.predictor import Predictor
from sagemaker.serializers import JSONSerializer
from sagemaker.deserializers import JSONDeserializer

class TextGenerationWithGuardrails:
    def __init__(self, endpoint_name: str, model_id: str, guardrail_id: str, guardrail_version: str, sagemaker_session=None):
        """
        Initialize the text generation class with guardrails.
        
        Args:
            endpoint_name: The SageMaker endpoint name
            model_id: The model ID (optional but useful for documentation)
            guardrail_id: The AWS Bedrock guardrail ID
            guardrail_version: The AWS Bedrock guardrail version
            sagemaker_session: SageMaker session object
        """
        # Create predictor directly instead of using retrieve_default
        self.predictor = Predictor(
            endpoint_name=endpoint_name,
            sagemaker_session=sagemaker_session,
            serializer=JSONSerializer(),
            deserializer=JSONDeserializer()
        )
        self.model_id = model_id
        self.bedrock_runtime = boto3.client('bedrock-runtime')
        self.guardrail_id = guardrail_id
        self.guardrail_version = guardrail_version

    def generate_text(self, inputs: str, max_new_tokens: int = 256, temperature: float = 0.0) -> str:
        """Generate text using the specified SageMaker endpoint."""
        payload = {
            "inputs": inputs,
            "parameters": {
                "max_new_tokens": max_new_tokens,
                "temperature": temperature,
                "stop": "<|eot_id|>"
            }
        }
    
        response = self.predictor.predict(payload)
        return response.get('generated_text', '')

    def analyze_text(self, grounding_source: str, query: str, guard_content: str, source: str) -> Tuple[bool, str, Dict[str, Any]]:
        """
        Analyze text using the ApplyGuardrail API with contextual grounding.
        Returns a tuple (passed, message, details) where:
        - passed is a boolean indicating if the guardrail passed,
        - message is either the guardrail message or an empty string,
        - details is a dictionary containing the full API response for further analysis if needed.
        """
        try:
            content = [
                {
                    "text": {
                        "text": grounding_source,
                        "qualifiers": ["grounding_source"]
                    }
                },
                {
                    "text": {
                        "text": query,
                        "qualifiers": ["query"]
                    }
                },
                {
                    "text": {
                        "text": guard_content,
                        "qualifiers": ["guard_content"]
                    }
                }
            ]

            response = self.bedrock_runtime.apply_guardrail(
                guardrailIdentifier=self.guardrail_id,
                guardrailVersion=self.guardrail_version,
                source=source,
                content=content
            )
            
            action = response.get("action", "")
            if action == "NONE":
                return True, "", response
            elif action == "GUARDRAIL_INTERVENED":
                message = response.get("outputs", [{}])[0].get("text", "Guardrail intervened")
                return False, message, response
            else:
                return False, f"Unknown action: {action}", response
        except ClientError as e:
            print(f"Error applying guardrail: {e}")
            raise

    def analyze_prompt(self, grounding_source: str, query: str) -> Tuple[bool, str, Dict[str, Any]]:
        """Analyze the input prompt."""
        return self.analyze_text(grounding_source, query, query, "INPUT")

    def analyze_output(self, grounding_source: str, query: str, generated_text: str) -> Tuple[bool, str, Dict[str, Any]]:
        """Analyze the generated output."""
        return self.analyze_text(grounding_source, query, generated_text, "OUTPUT")

    def generate_and_analyze(self, grounding_source: str, query: str, max_new_tokens: int = 256, temperature: float = 0.0) -> Tuple[bool, str, str]:
        """
        Generate text and analyze it with guardrails.
        Returns a tuple (passed, message, generated_text) where:
        - passed is a boolean indicating if the guardrail passed,
        - message is either the guardrail message or an empty string,
        - generated_text is the text generated by the model (if guardrail passed) or an empty string.
        """
        # First, analyze the prompt
        prompt_passed, prompt_message, _ = self.analyze_prompt(grounding_source, query)
        if not prompt_passed:
            return False, prompt_message, ""

        # If prompt passes, generate text
        generated_text = self.generate_text(query, max_new_tokens, temperature)

        # Analyze the generated text
        output_passed, output_message, _ = self.analyze_output(grounding_source, query, generated_text)
        if not output_passed:
            return False, output_message, ""

        return True, "", generated_text

### Now let's see a Sample Usage in action 

In [7]:
def main():
    import boto3
    import json
    import datetime
    from sagemaker.session import Session
    from botocore.exceptions import ClientError

    # Function to list available SageMaker endpoints
    def list_sagemaker_endpoints():
        try:
            sagemaker_client = boto3.client('sagemaker')
            response = sagemaker_client.list_endpoints()
            return response.get('Endpoints', [])
        except Exception as e:
            print(f"Error listing endpoints: {e}")
            return []

    # Get available SageMaker endpoints
    print("Fetching available SageMaker endpoints...")
    endpoints = list_sagemaker_endpoints()
    
    if not endpoints:
        print("No SageMaker endpoints found. Please create an endpoint first.")
        endpoint_name = input("Please enter your endpoint name manually: ")
        if not endpoint_name:
            print("No endpoint name provided. Exiting.")
            return
    else:
        # Display available endpoints
        print("\nAvailable SageMaker Endpoints:")
        for idx, endpoint in enumerate(endpoints):
            name = endpoint.get('EndpointName', 'Unknown')
            status = endpoint.get('EndpointStatus', 'Unknown')
            created_time = endpoint.get('CreationTime', 'Unknown')
            
            # Find endpoints containing "deepseek" in the name (likely to be deepseek model endpoints)
            is_deepseek = "deepseek" in name.lower()
            highlight = " (Likely Deepseek model)" if is_deepseek else ""
            
            print(f"{idx+1}. {name} - Status: {status}, Created: {created_time}{highlight}")
        
        # Try to automatically select a DeepSeek endpoint
        deepseek_endpoints = [ep for ep in endpoints if "deepseek" in ep.get('EndpointName', '').lower()]
        
        if deepseek_endpoints:
            # Sort by creation time (newest first) if available
            sorted_endpoints = sorted(
                deepseek_endpoints, 
                key=lambda x: x.get('CreationTime', datetime.datetime.min.replace(tzinfo=datetime.timezone.utc)),
                reverse=True
            )
            selected_endpoint = sorted_endpoints[0]
            endpoint_name = selected_endpoint.get('EndpointName')
            print(f"\nAutomatically selected Deepseek endpoint: {endpoint_name}")
        else:
            # Let user choose if no Deepseek endpoint found
            choice = input(f"\nEnter endpoint number (1-{len(endpoints)}) or name: ")
            
            try:
                idx = int(choice) - 1
                if 0 <= idx < len(endpoints):
                    endpoint_name = endpoints[idx].get('EndpointName')
                else:
                    print("Invalid choice, using the first endpoint.")
                    endpoint_name = endpoints[0].get('EndpointName')
            except ValueError:
                # If input is not a number, use it as the endpoint name
                endpoint_name = choice

    # Model ID for DeepSeek R1 Distill
    model_id = "deepseek-llm-r1-distill-llama-8b"
    
    # Create Bedrock client to access guardrails
    try:
        bedrock_client = boto3.client('bedrock')
        
        # List all available guardrails
        print("Fetching available guardrails...")
        guardrails_response = bedrock_client.list_guardrails()
        
        if 'guardrails' not in guardrails_response or not guardrails_response['guardrails']:
            print("No guardrails found. Please create a guardrail in AWS Bedrock first.")
            return

        # Display available guardrails
        print("\nAvailable Guardrails:")
        for idx, guardrail in enumerate(guardrails_response['guardrails']):
            guardrail_id = guardrail.get('id')
            name = guardrail.get('name', 'Unnamed')
            print(f"{idx+1}. {name} (ID: {guardrail_id})")
        
        # Use the first guardrail in the list
        selected_guardrail = guardrails_response['guardrails'][0]
        guardrail_id = selected_guardrail['id']
        guardrail_name = selected_guardrail.get('name', 'Unnamed')
        
        print(f"\nSelected guardrail: {guardrail_name} (ID: {guardrail_id})")
        
        # Get detailed information about the guardrail
        guardrail_info = bedrock_client.get_guardrail(
            guardrailIdentifier=guardrail_id,
            guardrailVersion='DRAFT'
        )
        
        print(f"Using guardrail version: DRAFT (Created: {guardrail_info.get('createdAt')})")
        
    except Exception as e:
        print(f"Error accessing guardrails: {e}")
        print("\nFalling back to manual guardrail ID entry...")
        
        # Prompt for guardrail ID if API access fails
        guardrail_id = input("Please enter your guardrail ID: ")
        if not guardrail_id:
            print("No guardrail ID provided. Exiting.")
            return
    
    guardrail_version = "DRAFT"  # Using DRAFT version as shown in your example
    
    # Create SageMaker session
    boto_session = boto3.Session()
    sagemaker_session = Session(boto_session)
    
    # Query and grounding information
    query = "What are is the Guarenteed Rate of Return for AB503 Product"
    grounding_source = "The AB503 Financial Product is currently offering a non-guaranteed rate of 7%"
    max_new_tokens = 512
    temperature = 0.0

    # Initialize TextGenerationWithGuardrails
    text_gen = TextGenerationWithGuardrails(
        endpoint_name=endpoint_name,
        model_id=model_id,
        guardrail_id=guardrail_id,
        guardrail_version=guardrail_version,
        sagemaker_session=sagemaker_session
    )

    # Bold text function
    def bold(text):
        return f"\033[1m{text}\033[0m"
    
    # Analyze input
    print(bold("\n=== Input Analysis ===\n"))
    input_passed, input_message, input_details = text_gen.analyze_prompt(grounding_source, query)
    if not input_passed:
        print(f"Input Guardrail Intervened. The response to the User is: {input_message}\n")
        print("Full API Response:")
        print(json.dumps(input_details, indent=2))
        print()
        return
    else:
        print("Input Prompt Passed The Guardrail Check - Moving to Generate the Response\n")

    # Generate text
    print(bold("\n=== Text Generation ===\n"))
    generated_text = text_gen.generate_text(query, max_new_tokens=max_new_tokens, temperature=temperature)
    print(f"Here is what the Model Responded with: {generated_text}\n")

    # Analyze output
    print(bold("\n=== Output Analysis ===\n"))
    print("Analyzing Model Response with the Response Guardrail\n")
    output_passed, output_message, output_details = text_gen.analyze_output(grounding_source, query, generated_text)
    if not output_passed:
        print(f"Output Guardrail Intervened. The response to the User is: {output_message}\n")
        print("Full API Response:")
        print(json.dumps(output_details, indent=2))
        print()
    else:
        print(f"Model Response Passed. The information presented to the user is: {generated_text}\n")

if __name__ == "__main__":
    main()

Fetching available SageMaker endpoints...

Available SageMaker Endpoints:
1. jumpstart-dft-deepseek-llm-r1-disti-20250406-215116 - Status: InService, Created: 2025-04-06 21:53:59.965000+00:00 (Likely Deepseek model)
2. DeepSeek-R1-Distill-Llama-8B-endpoint - Status: Creating, Created: 2025-04-06 21:49:57.181000+00:00 (Likely Deepseek model)
3. huggingface-pytorch-tgi-inference-2025-04-06-21-48-45-822 - Status: InService, Created: 2025-04-06 21:48:46.810000+00:00
4. DeepSeek-R1-Distill-Llama-8B-finetuned - Status: InService, Created: 2025-04-06 21:10:25.856000+00:00 (Likely Deepseek model)

Automatically selected Deepseek endpoint: jumpstart-dft-deepseek-llm-r1-disti-20250406-215116
Fetching available guardrails...

Available Guardrails:
1. fiduciary-advice-llm-guardrail (ID: p7x0618oz8n0)

Selected guardrail: fiduciary-advice-llm-guardrail (ID: p7x0618oz8n0)
Using guardrail version: DRAFT (Created: 2025-04-06 22:02:52.978652+00:00)

=== Input Analysis ===

Input Prompt Passed The Guard

#### Cleanup

In [ ]:
client.delete_guardrail(guardrailIdentifier="<replace with guardrail ID or ARN>")